In [15]:

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import tensorflow as tf
from pandas import Series
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Bidirectional
from keras.layers import Input
from keras import layers

import tensorflow as tf
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing import sequence

# fix random seed for reproducibility
tf.random.set_seed(7)

In [2]:
df = pd.read_csv(f'../../db/data/merge/secondary/logs_.csv', index_col=0)
df.reset_index(inplace=True, drop=True)

df

,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
0,-0.056357,0.044868,-0.236804,-0.027523,-0.191681,0.090711,-0.187507,-0.277763,-0.046156,-0.119630,...,-0.147436,0.041166,-0.099046,0.009273,0.090472,-0.190166,-0.139295,0.145630,0.228210,-0.048759
1,0.057418,0.116563,-0.162348,-0.068844,-0.031983,0.047432,-0.211379,-0.267380,-0.023087,0.033756,...,-0.180774,-0.117932,-0.052168,0.159828,0.212410,-0.054403,0.069672,0.087273,0.144556,-0.126891
2,-0.206473,0.062703,-0.262812,-0.018366,-0.243434,0.294601,-0.058796,-0.346427,0.000000,-0.215034,...,-0.255167,-0.003746,-0.240322,-0.190002,0.082882,-0.245215,-0.223152,0.033227,0.323370,-0.029303
3,-0.373553,-0.049272,0.125235,-0.404915,-0.733783,0.309505,0.504518,0.201297,0.063483,-0.320355,...,0.300982,-0.251275,-0.561166,0.052171,0.367330,0.544108,-0.307650,0.348187,0.669742,0.846226
4,-0.036356,-0.236578,-0.062598,0.101392,0.012918,-0.200263,-0.070237,0.133973,0.184438,0.191093,...,0.040733,0.270002,0.152371,-0.147307,-0.350272,-0.203694,-0.098087,-0.024836,-0.256038,-0.111437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111091,-0.111104,-0.019367,-0.046219,-0.060675,-0.152940,0.088497,0.060510,-0.024679,0.303152,0.193995,...,0.078071,0.051580,-0.035225,-0.039843,0.053424,0.017658,-0.083248,0.045766,0.133986,0.100968
111092,-0.111104,-0.019367,-0.046219,-0.060675,-0.152940,0.088497,0.060510,-0.024679,0.303152,0.193995,...,0.078071,0.051580,-0.035225,-0.039843,0.053424,0.017658,-0.083248,0.045766,0.133986,0.100968
111093,-0.111104,-0.019367,-0.046219,-0.060675,-0.152940,0.088497,0.060510,-0.024679,0.303152,0.193995,...,0.078071,0.051580,-0.035225,-0.039843,0.053424,0.017658,-0.083248,0.045766,0.133986,0.100968
111094,-0.111104,-0.019367,-0.046219,-0.060675,-0.152940,0.088497,0.060510,-0.024679,0.303152,0.193995,...,0.078071,0.051580,-0.035225,-0.039843,0.053424,0.017658,-0.083248,0.045766,0.133986,0.100968


In [12]:
#convert logs to classification states = ( 1, 0, -1 ) 

def condition(x, threshold):
    if x>threshold:
        return 1
    elif x<-threshold:
        return -1
    else:
        return 0

def binary(x):
    if x>0:
        return 1
    else:
        return -1


df_ = df.copy()

for sym in df.columns:
    df_[sym] = df[sym].map( lambda x: binary(x) )

df_.reset_index(inplace=True, drop=True)


y = df_.to_numpy()

y.shape

(111096, 28)

In [13]:
# load features

df2 = pd.read_csv('../../db/data/merge/tendency/tendency.csv', index_col=0)
df2.reset_index(inplace=True, drop=True)

df2

X = df2.to_numpy()

X.shape

(111096, 76)

In [25]:
X_train = np.array([X[1000:int(len(X)*0.5)]])
y_train = np.array([y[1000:int(len(y)*0.5)]])

X_train.shape, y_train.shape

((1, 54548, 76), (1, 54548, 28))

In [30]:
def initialize_model(X , y):

    model = Sequential()
    
    model.add(LSTM(50, activation='relu' , input_shape=(X.shape[1], X.shape[0])))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(50 , activation='relu'))
    model.add(Dense(y.shape[-1], activation='sigmoid',))


    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    return model


model = initialize_model(X_train , y_train)

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 50)                10400     
                                                                 
 dropout_8 (Dropout)         (None, 50)                0         
                                                                 
 dense_15 (Dense)            (None, 50)                2550      
                                                                 
 dense_16 (Dense)            (None, 28)                1428      
                                                                 
Total params: 14,378
Trainable params: 14,378
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100


ValueError: in user code:

    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 264, in assert_input_compatibility
        ra